নিম্নলিখিত নোটবুকটি GitHub Copilot Chat দ্বারা স্বয়ংক্রিয়ভাবে তৈরি করা হয়েছে এবং শুধুমাত্র প্রাথমিক সেটআপের জন্য ব্যবহৃত হবে


# প্রম্পট ইঞ্জিনিয়ারিং পরিচিতি
প্রম্পট ইঞ্জিনিয়ারিং হলো প্রাকৃতিক ভাষা প্রক্রিয়াকরণ কাজের জন্য প্রম্পট ডিজাইন ও অপ্টিমাইজ করার প্রক্রিয়া। এতে সঠিক প্রম্পট নির্বাচন, তাদের প্যারামিটার টিউন করা এবং তাদের কার্যকারিতা মূল্যায়ন করা অন্তর্ভুক্ত। প্রম্পট ইঞ্জিনিয়ারিং উচ্চ নির্ভুলতা ও দক্ষতা অর্জনের জন্য অত্যন্ত গুরুত্বপূর্ণ। এই অংশে, আমরা এক্সপ্লোরেশনের জন্য OpenAI মডেল ব্যবহার করে প্রম্পট ইঞ্জিনিয়ারিংয়ের মৌলিক বিষয়গুলো আলোচনা করব।


### অনুশীলন ১: টোকেনাইজেশন
tiktoken ব্যবহার করে টোকেনাইজেশন অন্বেষণ করুন, যা OpenAI-এর একটি ওপেন-সোর্স দ্রুত টোকেনাইজার
আরও উদাহরণের জন্য দেখুন [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst)


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### অনুশীলন ২: OpenAI API কী সেটআপ যাচাই করুন

নিচের কোডটি চালান যাতে আপনি নিশ্চিত হতে পারেন যে আপনার OpenAI এন্ডপয়েন্ট সঠিকভাবে সেটআপ হয়েছে। এই কোডটি শুধু একটি সহজ প্রম্পট দিয়ে চেষ্টা করে এবং কমপ্লিশন যাচাই করে। ইনপুট `oh say can you see` দিলে আউটপুট হওয়া উচিত `by the dawn's early light..` এর মতো কিছু।


In [ ]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

## Updated
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

### অনুশীলন ৩: মনগড়া তথ্য

দেখুন কী হয় যখন আপনি LLM-কে এমন একটি বিষয়ে উত্তর দিতে বলেন, যা হয়তো আসলেই নেই, অথবা এমন বিষয়ে যা সে জানে না কারণ সেটি তার প্রি-ট্রেইনড ডেটাসেটের বাইরে (আরও সাম্প্রতিক)। দেখুন, আপনি যদি ভিন্ন কোনো প্রম্পট দেন বা অন্য কোনো মডেল ব্যবহার করেন, তাহলে উত্তর কীভাবে পরিবর্তিত হয়।


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

### অনুশীলন ৪: নির্দেশনা ভিত্তিক
"text" ভেরিয়েবলটি ব্যবহার করে মূল বিষয়বস্তু নির্ধারণ করুন
এবং "prompt" ভেরিয়েবলটি ব্যবহার করে সেই মূল বিষয়বস্তুর সাথে সম্পর্কিত একটি নির্দেশনা দিন।

এখানে আমরা মডেলকে অনুরোধ করছি, টেক্সটটি দ্বিতীয় শ্রেণির শিক্ষার্থীর জন্য সংক্ষেপে উপস্থাপন করতে।


In [ ]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

### অনুশীলন ৫: জটিল প্রম্পট
একটি অনুরোধ চেষ্টা করুন যেখানে system, user এবং assistant বার্তা রয়েছে
System অ্যাসিস্ট্যান্টের প্রসঙ্গ নির্ধারণ করে
User এবং Assistant বার্তা বহু-পর্বের কথোপকথনের প্রসঙ্গ দেয়

লক্ষ্য করুন কিভাবে অ্যাসিস্ট্যান্টের ব্যক্তিত্ব "বিদ্রূপাত্মক" হিসেবে system প্রসঙ্গে নির্ধারণ করা হয়েছে।
ভিন্ন ব্যক্তিত্বের প্রসঙ্গ ব্যবহার করে দেখুন। অথবা ইনপুট/আউটপুট বার্তার ভিন্ন সিরিজ চেষ্টা করুন


In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

### অনুশীলন: আপনার অন্তর্দৃষ্টি অন্বেষণ করুন
উপরের উদাহরণগুলো আপনাকে কিছু প্যাটার্ন দেখিয়েছে, যেগুলো ব্যবহার করে আপনি নতুন প্রম্পট (সহজ, জটিল, নির্দেশনা ইত্যাদি) তৈরি করতে পারেন - চেষ্টা করুন আরও কিছু অনুশীলন তৈরি করতে, যাতে আমরা আলোচনা করা অন্যান্য ধারণা যেমন উদাহরণ, সংকেত এবং আরও অনেক কিছু অন্বেষণ করা যায়।



---

**দায়িত্ব অস্বীকার**:
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব নির্ভুলতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ভুল বা অসঙ্গতি থাকতে পারে। মূল ভাষায় রচিত নথিটিকেই চূড়ান্ত ও নির্ভরযোগ্য উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদের ব্যবহারে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
